In [70]:
#### Imports ####

# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install causality
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install decorator>=4.1.2
!{sys.executable} -m pip install networkx>=2.0
!{sys.executable} -m pip install numpy>=1.13.3
!{sys.executable} -m pip install pandas>=0.20.3
!{sys.executable} -m pip install patsy>=0.4.1
!{sys.executable} -m pip install python-dateutil>=2.6.1
!{sys.executable} -m pip install pytz>=2017.2
!{sys.executable} -m pip install scipy>=0.19.1
!{sys.executable} -m pip install six>=1.11.0
!{sys.executable} -m pip install statsmodels>=0.8.0

!{sys.executable} -m pip install networkx
!{sys.executable} -m pip install matplotlib
!{sys.executable} -m pip install pycm


# causality imports
# IC Algorithm Tutorial: https://pypi.org/project/causality/
import numpy
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import time


from causality.inference.search import IC
from causality.inference.independence_tests import RobustRegressionTest

from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve, roc_curve, auc, average_precision_score
from sklearn.metrics import confusion_matrix

from pycm import *


print('causality packages imported successfully')

You should consider upgrading via the '/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


You should consider upgrading via the '/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


You should consider upgrading via the '/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
causality packages imported successfully


In [190]:
import os

directory = '/Users/yaminimathur/Desktop/COM S 673 Project/Data without noise'
df_index = []
time_df = pd.DataFrame(columns=['Time Taken'], index = index_df_IC)

for filename in os.listdir(directory):
  if('Yeast' in filename or 'Ecoli' in filename):
    directory = directory+filename
    split_filename = (filename.split('-'))
    print(split_filename)
  
    if(split_filename[2] in ['null','nonoise']):
      type = split_filename[3].split('.')
      split_mix = split_filename[2]+'-'+type[0]
    else:
      type = split_filename[2].split('.')
      split_mix = type[0]

    df_index.append(split_filename[1]+'-'+split_mix)
    
df_columns = ['Accuracy']
print(df_index)
result_df = pd.DataFrame(columns=df_columns, index=df_index)
time_df = pd.DataFrame(columns=['Time Taken'], index = df_index)
result_df
time_df

['InSilicoSize10', 'Yeast2', 'nonoise', 'null', 'mutants.tsv']
['InSilicoSize10', 'Yeast2', 'nonoise', 'proteins', 'null', 'mutants.tsv']
['InSilicoSize10', 'Ecoli1', 'nonoise', 'proteins', 'heterozygous.tsv']
['InSilicoSize10', 'Ecoli1', 'nonoise', 'heterozygous.tsv']
['InSilicoSize10', 'Ecoli2', 'nonoise', 'proteins', 'trajectories.tsv']
['InSilicoSize10', 'Yeast1', 'nonoise', 'trajectories.tsv']
['InSilicoSize10', 'Ecoli1', 'nonoise', 'trajectories.tsv']
['InSilicoSize10', 'Yeast3', 'nonoise', 'proteins', 'null', 'mutants.tsv']
['InSilicoSize10', 'Ecoli1', 'nonoise', 'proteins', 'trajectories.tsv']
['InSilicoSize10', 'Yeast1', 'nonoise', 'heterozygous.tsv']
['InSilicoSize10', 'Yeast3', 'nonoise', 'null', 'mutants.tsv']
['InSilicoSize10', 'Yeast1', 'nonoise', 'proteins', 'null', 'mutants.tsv']
['InSilicoSize10', 'Ecoli2', 'nonoise', 'proteins', 'heterozygous.tsv']
['InSilicoSize10', 'Ecoli2', 'nonoise', 'null', 'mutants.tsv']
['InSilicoSize10', 'Yeast2', 'nonoise', 'proteins', 'traje

,Time Taken
Yeast2-nonoise-null,NaN
Yeast2-nonoise-proteins,NaN
Ecoli1-nonoise-proteins,NaN
Ecoli1-nonoise-heterozygous,NaN
Ecoli2-nonoise-proteins,NaN
Yeast1-nonoise-trajectories,NaN
Ecoli1-nonoise-trajectories,NaN
Yeast3-nonoise-proteins,NaN
Ecoli1-nonoise-proteins,NaN
Yeast1-nonoise-heterozygous,NaN


In [212]:
import numpy as np

header_dct = {'G1': 'c', 'G2': 'c', 'G3': 'c', 'G4': 'c', 'G5': 'c', 'G6': 'c', 'G7': 'c', 'G8': 'c', 'G9': 'c', 'G10': 'c'}

def print_theta_accuracy(standardnw_path,dataset_path):
  
  acc_value = []
  data = pd.read_csv(dataset_path, sep='\t') 
  data = data.iloc[: , 1:]


  benchmark_network = pd.read_csv(standardnw_path, sep='\t', header=None)
  benchmark_network = benchmark_network.loc[benchmark_network[2] == 1]


  benchmark_graph = nx.Graph()

  for i in (1,10):
      benchmark_graph.add_node('G'+str(i))

  for row in range(0,benchmark_network.shape[0]):
      benchmark_graph.add_edge(benchmark_network[0][row], benchmark_network[1][row])


  benchmark_adjMatrix = nx.adjacency_matrix(benchmark_graph)
  # Check time Elapsed for IC Algorithm to run
  # Time in seconds
  start = time.time()
  # run the IC search
  ic_algorithm = IC(RobustRegressionTest)
  predicted_graph = ic_algorithm.search(data, header_dct)
  end = time.time()
#   print(end - start)
  time_elapsed = end - start
#   time_list.append(time_elapsed)

  predicted_adjMatrix = nx.adjacency_matrix(predicted_graph)
  y_test = benchmark_adjMatrix.todense().flatten()
  y_pred = predicted_adjMatrix.todense().flatten()
  y_pred = np.asarray(y_pred)
  y_test = np.asarray(y_test)
  y_pred = y_pred.reshape(y_pred.shape[1],)
  y_test = y_test.reshape(y_test.shape[1],)
  cm = ConfusionMatrix(y_test, y_pred)
#   acc_list.append(cm.ACC_Macro)
  
  return time_elapsed,cm.ACC_Macro;

In [245]:
directory = '/Users/yaminimathur/Desktop/COM S 673 Project/Data without noise/'
goldstandard = '/Users/yaminimathur/Desktop/COM S 673 Project/DREAM3 gold standards/'
df_columns = ['accuracy','timetaken']
df_index = os.listdir(directory)
filename_df = []
#result_df = pd.DataFrame(columns = df_columns, index = filename_df)
result_df = pd.DataFrame(columns = df_columns, index= df_index)


for filename in os.listdir(directory):
    data_path = directory+filename
    
    filename_temp  = filename
    split_filename = (filename_temp.split('-'))
    if(split_filename[2] == 'null'):
      type = split_filename[3].split('.')
    else:
      type = split_filename[2].split('.')
      split_mix = type[0]
        
    print(filename_temp, split_filename)

    index_location = split_filename[1]+'-'+split_mix
    print(split_filename[1]+'-'+split_filename[2]+'-'+split_filename[3])
    filename_df = split_filename[1]+'-'+split_filename[2]+'-'+split_filename[3]
    print(filename_df)
    
    goldstandardname = ('DREAM3GoldStandard_'+split_filename[0]+'_'+split_filename[1]+'.txt')
    goldstandard_path = goldstandard+goldstandardname
    time_value, acc_value = print_theta_accuracy(goldstandard_path, data_path)
    print(filename+':')
    print(time_value, acc_value)
    print()
    result_df['accuracy'][filename] = acc_value
    result_df['timetaken'][filename] = time_value
    

InSilicoSize10-Yeast2-nonoise-null-mutants.tsv ['InSilicoSize10', 'Yeast2', 'nonoise', 'null', 'mutants.tsv']
Yeast2-nonoise-null
Yeast2-nonoise-null
InSilicoSize10-Yeast2-nonoise-null-mutants.tsv:
6.021622896194458 0.54

InSilicoSize10-Yeast2-nonoise-proteins-null-mutants.tsv ['InSilicoSize10', 'Yeast2', 'nonoise', 'proteins', 'null', 'mutants.tsv']
Yeast2-nonoise-proteins
Yeast2-nonoise-proteins
InSilicoSize10-Yeast2-nonoise-proteins-null-mutants.tsv:
4.611642122268677 0.54

InSilicoSize10-Ecoli1-nonoise-proteins-heterozygous.tsv ['InSilicoSize10', 'Ecoli1', 'nonoise', 'proteins', 'heterozygous.tsv']
Ecoli1-nonoise-proteins
Ecoli1-nonoise-proteins


/opt/anaconda3/lib/python3.7/site-packages/statsmodels/robust/robust_linear_model.py:290: ConvergenceWarning: Estimated scale is 0.0 indicating that the most last iteration produced a perfect fit of the weighted data.
  'weighted data.', ConvergenceWarning)
/opt/anaconda3/lib/python3.7/site-packages/statsmodels/robust/robust_linear_model.py:290: ConvergenceWarning: Estimated scale is 0.0 indicating that the most last iteration produced a perfect fit of the weighted data.
  'weighted data.', ConvergenceWarning)
/opt/anaconda3/lib/python3.7/site-packages/statsmodels/robust/robust_linear_model.py:290: ConvergenceWarning: Estimated scale is 0.0 indicating that the most last iteration produced a perfect fit of the weighted data.
  'weighted data.', ConvergenceWarning)
/opt/anaconda3/lib/python3.7/site-packages/statsmodels/robust/robust_linear_model.py:290: ConvergenceWarning: Estimated scale is 0.0 indicating that the most last iteration produced a perfect fit of the weighted data.
  'weigh

/opt/anaconda3/lib/python3.7/site-packages/statsmodels/robust/robust_linear_model.py:290: ConvergenceWarning: Estimated scale is 0.0 indicating that the most last iteration produced a perfect fit of the weighted data.
  'weighted data.', ConvergenceWarning)
/opt/anaconda3/lib/python3.7/site-packages/statsmodels/robust/robust_linear_model.py:290: ConvergenceWarning: Estimated scale is 0.0 indicating that the most last iteration produced a perfect fit of the weighted data.
  'weighted data.', ConvergenceWarning)
/opt/anaconda3/lib/python3.7/site-packages/statsmodels/robust/robust_linear_model.py:290: ConvergenceWarning: Estimated scale is 0.0 indicating that the most last iteration produced a perfect fit of the weighted data.
  'weighted data.', ConvergenceWarning)
/opt/anaconda3/lib/python3.7/site-packages/statsmodels/robust/robust_linear_model.py:290: ConvergenceWarning: Estimated scale is 0.0 indicating that the most last iteration produced a perfect fit of the weighted data.
  'weigh

/opt/anaconda3/lib/python3.7/site-packages/statsmodels/robust/robust_linear_model.py:290: ConvergenceWarning: Estimated scale is 0.0 indicating that the most last iteration produced a perfect fit of the weighted data.
  'weighted data.', ConvergenceWarning)
/opt/anaconda3/lib/python3.7/site-packages/statsmodels/robust/robust_linear_model.py:290: ConvergenceWarning: Estimated scale is 0.0 indicating that the most last iteration produced a perfect fit of the weighted data.
  'weighted data.', ConvergenceWarning)
/opt/anaconda3/lib/python3.7/site-packages/statsmodels/robust/robust_linear_model.py:290: ConvergenceWarning: Estimated scale is 0.0 indicating that the most last iteration produced a perfect fit of the weighted data.
  'weighted data.', ConvergenceWarning)
/opt/anaconda3/lib/python3.7/site-packages/statsmodels/robust/robust_linear_model.py:290: ConvergenceWarning: Estimated scale is 0.0 indicating that the most last iteration produced a perfect fit of the weighted data.
  'weigh

InSilicoSize10-Ecoli1-nonoise-proteins-heterozygous.tsv:
21.31676411628723 0.68

InSilicoSize10-Ecoli1-nonoise-heterozygous.tsv ['InSilicoSize10', 'Ecoli1', 'nonoise', 'heterozygous.tsv']
Ecoli1-nonoise-heterozygous.tsv
Ecoli1-nonoise-heterozygous.tsv


...

/opt/anaconda3/lib/python3.7/site-packages/statsmodels/robust/robust_linear_model.py:290: ConvergenceWarning: Estimated scale is 0.0 indicating that the most last iteration produced a perfect fit of the weighted data.
  'weighted data.', ConvergenceWarning)
/opt/anaconda3/lib/python3.7/site-packages/statsmodels/robust/robust_linear_model.py:290: ConvergenceWarning: Estimated scale is 0.0 indicating that the most last iteration produced a perfect fit of the weighted data.
  'weighted data.', ConvergenceWarning)
/opt/anaconda3/lib/python3.7/site-packages/statsmodels/robust/robust_linear_model.py:290: ConvergenceWarning: Estimated scale is 0.0 indicating that the most last iteration produced a perfect fit of the weighted data.
  'weighted data.', ConvergenceWarning)
/opt/anaconda3/lib/python3.7/site-packages/statsmodels/robust/robust_linear_model.py:290: ConvergenceWarning: Estimated scale is 0.0 indicating that the most last iteration produced a perfect fit of the weighted data.
  'weigh

InSilicoSize10-Ecoli1-nonoise-null-mutants.tsv:
4.678889989852905 0.74

InSilicoSize10-Yeast2-nonoise-heterozygous.tsv ['InSilicoSize10', 'Yeast2', 'nonoise', 'heterozygous.tsv']
Yeast2-nonoise-heterozygous.tsv
Yeast2-nonoise-heterozygous.tsv
InSilicoSize10-Yeast2-nonoise-heterozygous.tsv:
6.242058038711548 0.34

InSilicoSize10-Yeast2-nonoise-proteins-heterozygous.tsv ['InSilicoSize10', 'Yeast2', 'nonoise', 'proteins', 'heterozygous.tsv']
Yeast2-nonoise-proteins
Yeast2-nonoise-proteins
InSilicoSize10-Yeast2-nonoise-proteins-heterozygous.tsv:
6.184625148773193 0.36

InSilicoSize10-Ecoli1-nonoise-proteins-null-mutants.tsv ['InSilicoSize10', 'Ecoli1', 'nonoise', 'proteins', 'null', 'mutants.tsv']
Ecoli1-nonoise-proteins
Ecoli1-nonoise-proteins


/opt/anaconda3/lib/python3.7/site-packages/statsmodels/robust/robust_linear_model.py:290: ConvergenceWarning: Estimated scale is 0.0 indicating that the most last iteration produced a perfect fit of the weighted data.
  'weighted data.', ConvergenceWarning)
/opt/anaconda3/lib/python3.7/site-packages/statsmodels/robust/robust_linear_model.py:290: ConvergenceWarning: Estimated scale is 0.0 indicating that the most last iteration produced a perfect fit of the weighted data.
  'weighted data.', ConvergenceWarning)
/opt/anaconda3/lib/python3.7/site-packages/statsmodels/robust/robust_linear_model.py:290: ConvergenceWarning: Estimated scale is 0.0 indicating that the most last iteration produced a perfect fit of the weighted data.
  'weighted data.', ConvergenceWarning)
/opt/anaconda3/lib/python3.7/site-packages/statsmodels/robust/robust_linear_model.py:290: ConvergenceWarning: Estimated scale is 0.0 indicating that the most last iteration produced a perfect fit of the weighted data.
  'weigh

/opt/anaconda3/lib/python3.7/site-packages/statsmodels/robust/robust_linear_model.py:290: ConvergenceWarning: Estimated scale is 0.0 indicating that the most last iteration produced a perfect fit of the weighted data.
  'weighted data.', ConvergenceWarning)
/opt/anaconda3/lib/python3.7/site-packages/statsmodels/robust/robust_linear_model.py:290: ConvergenceWarning: Estimated scale is 0.0 indicating that the most last iteration produced a perfect fit of the weighted data.
  'weighted data.', ConvergenceWarning)
/opt/anaconda3/lib/python3.7/site-packages/statsmodels/robust/robust_linear_model.py:290: ConvergenceWarning: Estimated scale is 0.0 indicating that the most last iteration produced a perfect fit of the weighted data.
  'weighted data.', ConvergenceWarning)
/opt/anaconda3/lib/python3.7/site-packages/statsmodels/robust/robust_linear_model.py:290: ConvergenceWarning: Estimated scale is 0.0 indicating that the most last iteration produced a perfect fit of the weighted data.
  'weigh

InSilicoSize10-Ecoli1-nonoise-proteins-null-mutants.tsv:
5.129029989242554 0.74

InSilicoSize10-Yeast3-nonoise-trajectories.tsv ['InSilicoSize10', 'Yeast3', 'nonoise', 'trajectories.tsv']
Yeast3-nonoise-trajectories.tsv
Yeast3-nonoise-trajectories.tsv
InSilicoSize10-Yeast3-nonoise-trajectories.tsv:
6.162584066390991 0.66



In [246]:
result_df

,accuracy,timetaken
InSilicoSize10-Yeast2-nonoise-null-mutants.tsv,0.54,6.021623
InSilicoSize10-Yeast2-nonoise-proteins-null-mutants.tsv,0.54,4.611642
InSilicoSize10-Ecoli1-nonoise-proteins-heterozygous.tsv,0.68,21.316764
InSilicoSize10-Ecoli1-nonoise-heterozygous.tsv,0.68,17.524494
InSilicoSize10-Ecoli2-nonoise-proteins-trajectories.tsv,0.68,4.660142
InSilicoSize10-Yeast1-nonoise-trajectories.tsv,0.62,12.232051
InSilicoSize10-Ecoli1-nonoise-trajectories.tsv,0.72,6.740209
InSilicoSize10-Yeast3-nonoise-proteins-null-mutants.tsv,0.48,3.388045
InSilicoSize10-Ecoli1-nonoise-proteins-trajectories.tsv,0.7,4.76551
InSilicoSize10-Yeast1-nonoise-heterozygous.tsv,0.7,22.832815
